In [1]:
import pathlib
import os
os.chdir(pathlib.Path(os.getcwd()).parent)
from dnn.model_pipeline import UNetPipeline, get_masked_daily_product
from dnn.model_pipeline import ndvi, get_cdl, isin, crops_list
import descarteslabs as dl
import numpy as np
from descarteslabs.client.services import Places
import matplotlib as mpl
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import shapely
import shapely.ops
import shapely.prepared
import rasterio.features
import ipyleaflet
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

/opt/conda/lib/python3.7/site-packages/pkg_resources/__init__.py:1151: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  self, resource_name
/opt/conda/lib/python3.7/site-packages/pkg_resources/__init__.py:1151: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  self, resource_name
/opt/conda/lib/python3.7/site-packages/ipyleaflet/leaflet.py:338: DeprecationWarning: metadata {'dtype': None} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  data = Dataset().tag(dtype=None, sync=True, to_json=ds_x_to_json)


### Generate the Tilated Image Collection and Get Rid of Invalid Tiles

In [2]:
# Create the Central Valley AOI and Tilation
sac = shapely.geometry.shape(
    dl.places.shape("north-america_united-states_california_sacramento-valley").geometry
)
sj = shapely.geometry.shape(
    dl.places.shape("north-america_united-states_california_san-joaquin-valley").geometry)
central_valley_aoi = sac.union(sj)

tiles = dl.scenes.DLTile.from_shape(
    central_valley_aoi, resolution=20, tilesize=64, pad=0)
print(f'Number of tiles: {len(tiles)}')

Number of tiles: 62635


In [3]:
# Create Relevant Image Collections from our AOI / Tiles
start_datetime = "2019-01-01"
end_datetime = "2020-01-01"
# Create Landsat Image Collection
l8_daily = get_masked_daily_product(
    "landsat:LC08:01:T1:TOAR", start_datetime, end_datetime
).pick_bands("red green blue nir swir1")
l8_daily = l8_daily.concat_bands(ndvi(l8_daily))
# Create CDL Image Collection
cdl = get_cdl(start_date="2017-01-01", end_date="2020-01-01")
is_crops = isin(cdl, crops_list)
is_crops_19 = is_crops[-1]
four_year_combo = is_crops.sum(axis="images") + is_crops_19  # double-weight 2019
four_year_binary = four_year_combo >= 2
cdl_mask = ~four_year_binary

In [4]:
# Find the the valid tiles
central_valley_ctx = dl.scenes.AOI(central_valley_aoi, shape=(2048, 2048), crs="EPSG:4326")
all_cdl = four_year_binary.compute(central_valley_ctx)
all_cdl.geocontext["gdal_geotrans"]
shapes = list(
    geom for geom, value in
    rasterio.features.shapes(
        all_cdl.ndarray.astype("uint8"), 
        transform=rasterio.transform.Affine.from_gdal(*all_cdl.geocontext["gdal_geotrans"])
    )
    if value == 1
)
print(f"length of shapes: {len(shapes)}")
all_valid = shapely.ops.unary_union([shapely.geometry.shape(s) for s in shapes]).simplify(0.3)
print(f'Type of all_valid: {type(all_valid)}')
all_valid_prepped = shapely.prepared.prep(all_valid)
valid_tiles = [t for t in tqdm(tiles) if all_valid_prepped.intersects(t.geometry)]
print(f'No. Valid Tiles: {len(valid_tiles)}')
print(f'Percentage of valid tiles: {100*(len(valid_tiles) / len(tiles))}')


Job ID: 43c2c4fe1f41ef42a9a99ed78e9a1f9bba6f9bd775f35dda
[######] | Steps: 10/10 | Stage: SUCCEEDED                                    
Job ID: 43c2c4fe1f41ef42a9a99ed78e9a1f9bba6f9bd775f35dda
length of shapes: 4957
Type of all_valid: <class 'shapely.geometry.multipolygon.MultiPolygon'>



No. Valid Tiles: 19893
Percentage of valid tiles: 31.76019797237966


## Train the U-Net Model

In [5]:
unet_params = {
    'img_height': 64,
    'img_width': 64,
    'bands': 6,
    'time_steps': 12,
    'nclasses': len(crops_list) + 1
}
random_seed = 2020
L = len(valid_tiles)
test_split = np.random.RandomState(random_seed)
tr_ix = test_split.choice(np.arange(L), int(0.85 * L), replace=False)
tst_ix = np.array([k for k in np.arange(L) if k not in tr_ix])
u_net = UNetPipeline(model_params=unet_params, 
                     tiles=np.asanyarray(valid_tiles),
                     img_prod_id="landsat:LC08:01:T1:TOAR",
                     train_ix=tr_ix,
                     test_ix=tst_ix,
                     random_seed=random_seed
                     )

In [ ]:
u_net.train_model(16, 5)

Starting Training ...
Year 2017
Epoch 1 / 5: 
Batch 1 out of 1057
################################
Train on 16 samples
16/16 [==============================] - 13s 816ms/sample - loss: 7.6772 - accuracy: 0.0207
Year 2017
Epoch 1 / 5: 
Batch 11 out of 1057
################################
Train on 16 samples
16/16 [==============================] - 2s 95ms/sample - loss: 9.4036 - accuracy: 0.0203
Year 2017
Epoch 1 / 5: 
Batch 21 out of 1057
################################
Train on 16 samples
16/16 [==============================] - 2s 94ms/sample - loss: 7.6489 - accuracy: 0.0219
Year 2017
Epoch 1 / 5: 
Batch 31 out of 1057
################################
Train on 16 samples
16/16 [==============================] - 1s 93ms/sample - loss: 9.4945 - accuracy: 0.0214
Year 2017
Epoch 1 / 5: 
Batch 41 out of 1057
################################
Train on 16 samples
16/16 [==============================] - 2s 95ms/sample - loss: 8.6692 - accuracy: 0.0209
Year 2017
Epoch 1 / 5: 
Batch 51 out 